# Multidimensional Least-Squares Khatri-Rao Factorization (MLS-KRF)

In [62]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot
from numpy.random import randn, rand, randint
from numpy.linalg import svd, matrix_rank, norm
from scipy.io import loadmat
import time

krf_m = loadmat("krf_matrix_3D.mat")

## Problem 1

In [72]:
def error(X_0, X_0_hat):
    return norm(X_0_hat - X_0, 'fro')**2/norm(X_0, 'fro')**2

def Hermitian(X):
    return np.array(np.matrix(X).H)

def HOSVD(X, ml_rank=None):
    U_list = []
    S = X
    
    for n in range(X.ndim):
        X_n = tl.unfold(X, n)
        Un, _, _ = svd(X_n)
        
        if ml_rank:
            Un = Un[:, :ml_rank[n]]

        U_list.append(Un)

        S = mode_dot(S, Hermitian(Un), n)
        
    return S, U_list

def khatri_rao(A, B):
    assert A.shape[1] == B.shape[1]
    
    A = A.T
    B = B.T

    result = []

    for Ai in range(0, len(A)):
        for Bi in range(0, len(B)):
            if Ai == Bi:
                result.append(np.kron(A[Ai], B[Bi]))

    return np.array(result).T 

In [55]:
A = randn(3, 3, 3) + randn(3, 3, 3)*1j
X = khatri_rao(khatri_rao(tl.unfold(A, 0), tl.unfold(A, 1)), tl.unfold(A, 2))

In [57]:
def MLS_KRF(X, I):
    R = X.shape[1]
    Ir = I
    Ir.reverse()
    N = len(I)
    As = []

    A_list = []
    for r in range(R):
        xr = X[:, r]
        Xr = np.reshape(xr, Ir).T

        Sr, Ur = HOSVD(Xr)

        Ar = []
        for n in range(N):
            ur = Ur[N-1-n]
            ar = np.array((Sr.flat[0]**(1/N))*ur[:, 0])
            Ar.append(ar)

        A_list.append(Ar)

    for n in range(N):
        An = []
        for r in range(R):
            An.append(np.array(A_list)[r, n])       
        As.append(np.array(An).T)
        
    return As

In [58]:
As = MLS_KRF(X, [3, 3, 3])

assert X.all() == khatri_rao(khatri_rao(As[0], As[1]), As[2]).all()

In [59]:
tl.unfold(A, 0)

array([[-2.10156248-0.43467015j,  0.04360656-0.76543334j,
        -1.38634519-0.989357j  , -0.82418307-2.32548242j,
         0.1732303 +1.63278445j, -0.95375802+0.48884145j,
         1.33897861-0.2760527j ,  0.68680827+0.84377387j,
        -0.21902758-0.5229953j ],
       [ 0.76908327+0.69340358j, -1.81288363-1.22885066j,
         0.04319317+1.27632263j,  1.38987374+0.045241j  ,
        -1.23068461-0.34038785j, -0.49555198+0.12324824j,
        -0.23722306-0.1039959j , -1.4257949 +0.78575071j,
         1.79126996-0.45764284j],
       [-0.69022544-0.53319101j, -0.6922268 -1.42304997j,
         0.27633804+1.05982805j, -0.24809201+1.13117347j,
         0.59447185-1.40370162j,  0.11557326+0.59808823j,
        -1.48797064+0.58445952j, -1.81123441-0.12642544j,
         1.11279286-0.70107654j]])

In [60]:
As[0]

array([[-2.4405254 -0.5047785j , -0.5984034 +0.40868194j,
        -1.13822395+0.96213043j, -1.22879743-1.54592583j,
        -1.20845013-1.01328323j, -1.34304508+0.8327581j ,
        -1.07143781+0.86020607j, -0.68226953+0.74643287j,
        -0.5185167 +0.18457398j],
       [ 0.89312942+0.80524327j,  0.12495865+2.06627502j,
         1.091685  +0.2388419j ,  1.06385884-0.32729536j,
         1.07776582-0.58526401j, -0.71349252+0.24054125j,
         0.2598526 -0.01561083j, -0.91668725-1.20394182j,
        -0.34863619-1.76042759j],
       [-0.80155253-0.61918987j, -0.66664255+1.33894831j,
         0.95841074-0.00559886j,  0.10644906+0.92079759j,
         0.52497141+1.36679268j,  0.24155557+0.86510126j,
         1.06347933-1.20434863j, -0.13047308-1.68259317j,
        -0.62053909-1.11572008j]])

## Problem 2

In [79]:
I1, I2, I3 = 2, 3, 4
R = 4

SNRdb_range = [0, 5, 10, 15, 20, 25, 30]
No_experiments = 1000

NMSE = []

for SNRdb in SNRdb_range:
    exp_errors = [] # list of errors for the experiments
    
    for i in range(No_experiments):
        A = randn(I1, R) + randn(I1, R)*1j
        B = randn(I2, R) + randn(I2, R)*1j
        C = randn(I3, R) + randn(I3, R)*1j
        
        X_0 = khatri_rao(khatri_rao(A, B), C)
        
        V = randn(I1*I2*I3, R) + randn(I1*I2*I3, R)*1j
        
        # calculate alpha and X
        alpha = np.sqrt((1/10**(SNRdb/10))*(norm(X_0, 'fro')**2/norm(V, 'fro')**2))
        X = X_0 + alpha*V
        
        # Estimante via MLS-KRF
        An = MLS_KRF(X, [I1, I2, I3])
        
        X_0_hat = khatri_rao(khatri_rao(An[0], An[1]), An[2])
        
        exp_errors.append(error(X_0, X_0_hat))
        
    NMSE.append(np.sum(exp_errors)/No_experiments)

C:\Users\saulo_mendes\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [80]:
NMSE

[0.730116422306495,
 0.43993418645091165,
 0.3568665454279801,
 0.3330405215367379,
 0.3197915225332505,
 0.3142215656734646,
 0.31843292671644974]

In [81]:
import plotly.graph_objects as go

fig = go.Figure(
    layout=go.Layout(
        title="NMSE vs SNR curve",
        template="simple_white"))

fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE, mode='lines+markers', name='MLS-KRF'))

fig.update_yaxes(title='NMSE',showgrid=True,type="log")
fig.update_xaxes(title='SNR(dB)')

fig.show()